In [1]:
import ccxt
import pandas as pd
import numpy as np
import requests
import time
from datetime import datetime as dt ,timedelta
import pytz
import talib as ta
import seaborn as sns

import pandas_ta as ta

### Exchange Setting ################################################################################
ex = ccxt.ftx({'enableRateLimit':True})
ex_b = ccxt.binance({'enableRateLimit':True})
##############################################################################






def get_data(symbols,timeframe='1d',since=None,limit=1000,ex_mode='ftx'):
        if ex_mode =='ftx':
            ex = ccxt.ftx({'enableRateLimit':True})
        elif ex_mode =='binance':
            ex = ccxt.binance({'enableRateLimit':True})
        data= pd.DataFrame(ex.fetch_ohlcv(symbols,timeframe,since=None,limit=limit)
                         ,columns=['date','open','high','low','close','volume'])
        data['volume'] =(data['volume'].astype('float'))
        data['date']=data['date'].apply(ex.iso8601)
        data['date']=pd.to_datetime(data['date'])
        #   df['date']=df['date'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M'))
        #   df['date']=pd.to_datetime(df['date'])
        data.set_index('date',inplace=True)
        return data
    
    

In [159]:
# # Delete bull ,bear , future ,stock product
# # Create ALL SYMBOLS IN EXCHANGES FTX

# market_name = ex.load_markets().keys()
# symbol_list = list(market_name)
# symbol_lists = []
# ban_list = ['BULL','BEAR','HEDGE','HALF']



# for i in (range(len(symbol_list))):
#     if ('PERP' in symbol_list[i]) & ('USDT' not in symbol_list[i]) :
#         if not any(ban_sym  in symbol_list[i] for ban_sym in ban_list) :
#                         symbol_lists.append(symbol_list[i])
# print(len(symbol_lists))


In [24]:
symbol_lists = ['BTC-PERP','ETH-PERP','LUNA-PERP','BNB-PERP','SOL-PERP','AVAX-PERP']

In [118]:
# Create INDEX BENCHMARKS DATAFRAME
index_df = get_data('BTC-PERP').drop(columns='volume')
index_df['pct'] = np.log(index_df['close']/index_df['close'].shift(1)).fillna(0)
index_return = (((index_df['pct'] + 1).cumprod()-1))
last_index_return =index_return.iloc[-1]

In [183]:
df_all = pd.DataFrame()
dict_all = dict()
export_list= []
for symbol in symbol_lists:
        df =get_data(symbol).drop(columns='volume')
        df['symbol'] = symbol
        df['pct'] = np.log(df['close']/df['close'].shift(1)).fillna(0)
        
        # Calculate return 
        ret= df['pct'].mean()
        ann_ret = ret *252
        m_ret =ret*30
        ret = round(ret,3)
        m_ret = round(m_ret,3)
        ann_ret = round(ann_ret,3)
#         corr = df['pct'].corr(index_df['pct'])
        # last values
        stock_return = (((df['pct'] + 1).cumprod())-1).iloc[-1]
        returns_multiple = round((stock_return / last_index_return), 2)

        ema = [10, 20, 60]
        for x in ema: # LOOP TO EMA 
            df["EMA_"+str(x)] = df.ta.ema(x)
###############################################
        df['rsi_14']= df.ta.rsi(length=14)
        df['roc_14'] = df.ta.roc(length=14)
        df['roc_30'] = df.ta.roc(length=30)
        df['low_of_52week'] = df['close'].rolling(252).min()
        df['high_of_52week'] = df['close'].rolling(252).max()
                                
        # Storing required values 
        currentClose = df["close"].iloc[-1]
        # Clean format date
        first_day = df.index[0].strftime('%Y/%m/%d')
        last_day = df.index[-1].strftime('%Y/%m/%d')

        # print(first_day)
        moving_average_10 = df["EMA_10"].iloc[-1]
        moving_average_20 = df["EMA_20"].iloc[-1]
        moving_average_60 = df["EMA_60"].iloc[-1]
        rsi_14 = df['rsi_14'].iloc[-1]
        roc_14 = df["roc_14"].iloc[-1]
        roc_30 = df["roc_30"].iloc[-1]
        low_of_52week = df['low_of_52week'].iloc[-1]
        high_of_52week = df['high_of_52week'].iloc[-1]
        
        # Add DataFrame To ALL DATAFRAME
        df_all   = df_all.append(df)                            
        dict_all[symbol] = df                                
        # Condition Require
        try:
            # rolling EMA50
            moving_average_60_20 = df["EMA_60"].shift(20).iloc[-1]
        except Exception:
            moving_average_60_20 = np.nan

        # Condition 1: Current Price > 20 SMA and > 50 SMA
        condition_1 = currentClose > moving_average_20 > moving_average_60
        
        # Condition 2: 20 SMA and > 50 SMA
        condition_2 = moving_average_20 > moving_average_60        
        

        # Condition 3: 50 EMA trending up for at least 1 month
        condition_3 = moving_average_60 > moving_average_60_20        
        
        # Condition 4 : Current Price is at least 30% above 52 week low
        condition_4 = currentClose >= (1.3*low_of_52week)
        
        # Condition 5: Current Price is within 25% of 52 week high
        condition_5 = currentClose >= (.75*high_of_52week)        
        
        # If all conditions above are true, add stock to exportList
        if (condition_1 and condition_2 and condition_3 and condition_4):
            # Create  Dictionary append to list
            print (symbol + " made the Sahing requirements")
            details = {'symbol':symbol,'first_day':first_day,'last_day':last_day,'current_close':currentClose,'ema_20':moving_average_20,'ema_60':moving_average_60,'low_of_52week':low_of_52week,'high_of_52week':high_of_52week,
                       'cum_ret':stock_return,'return_compare_index':returns_multiple,'monthly_ret':m_ret}
            export_list.append(details)
        else :
            print (symbol + " Not  made the Sahing requirements")


BTC-PERP Not  made the Sahing requirements
ETH-PERP made the Sahing requirements
LUNA-PERP made the Sahing requirements
BNB-PERP made the Sahing requirements
SOL-PERP made the Sahing requirements
AVAX-PERP Not  made the Sahing requirements


In [181]:
dict_all['SOL-PERP']

,open,high,low,close,symbol,pct,EMA_10,EMA_20,EMA_60,rsi_14,roc_14,roc_30,low_of_52week,high_of_52week
date,,,,,,,,,,,,,,
2020-07-27 00:00:00+00:00,1.56625,2.0675,1.56375,1.7975,SOL-PERP,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-07-28 00:00:00+00:00,1.79750,1.8725,1.60750,1.7925,SOL-PERP,-0.002786,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-07-29 00:00:00+00:00,1.79250,2.1175,1.58250,1.6650,SOL-PERP,-0.073786,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-07-30 00:00:00+00:00,1.66500,1.8000,1.62500,1.6950,SOL-PERP,0.017858,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-07-31 00:00:00+00:00,1.69500,1.7000,1.49250,1.5350,SOL-PERP,-0.099152,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-02 00:00:00+00:00,134.57750,143.6425,131.50000,132.5425,SOL-PERP,-0.015237,116.291179,106.754583,104.877022,76.273914,43.452027,38.621032,27.9075,258.695
2022-04-03 00:00:00+00:00,132.54250,141.2625,130.62750,136.9275,SOL-PERP,0.032548,120.043237,109.628194,105.927858,78.026088,54.467257,55.026889,27.9075,258.695
2022-04-04 00:00:00+00:00,136.92750,138.3800,125.73500,132.1500,SOL-PERP,-0.035514,122.244467,111.773128,106.787600,71.804270,49.161917,47.427137,27.9075,258.695


In [137]:
# End LOOP        
df_all = df_all.reset_index()
df_all = df_all.set_index('symbol')
df_all = df_all.sort_values('date')

In [138]:
# Sort Symbols
df_all.loc[(df_all.index=='BTC-PERP')]

,date,open,high,low,close,pct,EMA_10,EMA_20,EMA_60,rsi_14,roc_14,roc_30,low_of_52week,high_of_52week
symbol,,,,,,,,,,,,,,
BTC-PERP,2019-07-20 00:00:00+00:00,10564.25,11108.75,10385.25,10765.75,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BTC-PERP,2019-07-21 00:00:00+00:00,10765.75,10834.00,10322.00,10588.75,-0.016578,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BTC-PERP,2019-07-22 00:00:00+00:00,10588.75,10688.25,10059.00,10321.50,-0.025563,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BTC-PERP,2019-07-23 00:00:00+00:00,10321.50,10323.25,9776.50,9841.75,-0.047596,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BTC-PERP,2019-07-24 00:00:00+00:00,9835.25,9915.75,9517.75,9770.00,-0.007317,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BTC-PERP,2022-04-02 00:00:00+00:00,46298.00,47234.00,45612.00,45821.00,-0.010356,45501.159194,44143.386331,42643.628270,59.922845,8.482883,7.905520,35072.0,67657.0
BTC-PERP,2022-04-03 00:00:00+00:00,45821.00,47499.00,45550.00,46425.00,0.013096,45669.130250,44360.682871,42767.607671,61.889553,12.436425,18.585404,35072.0,67657.0
BTC-PERP,2022-04-04 00:00:00+00:00,46425.00,46920.00,45125.00,46613.00,0.004041,45840.742932,44575.189264,42893.686108,62.506300,13.673609,18.319119,35072.0,67657.0


In [135]:
# View Symbols to meet conditions
df_req = pd.DataFrame(export_list)
df_req.sort_values('cum_ret',ascending=False)

,symbol,first_day,last_day,ema_20,ema_60,low_of_52week,high_of_52week,cum_ret,return_compare_index,monthly_ret
3,SOL-PERP,2020/07/27,2022/04/06,113.654216,107.787650,31.2525,31.2525,6.846806,9.43,0.203
0,ETH-PERP,2019/07/20,2022/04/06,3228.062476,3058.851039,2384.0000,2384.0000,2.140974,2.95,0.080
1,LUNA-PERP,2021/02/18,2022/04/06,102.665004,87.785331,10.7400,10.7400,2.015554,2.78,0.203
2,BNB-PERP,2019/07/20,2022/04/06,424.778548,415.262616,317.0950,317.0950,1.167884,1.61,0.079


In [143]:
df_all_test = df_all.reset_index()
df_all_test =df_all_test.set_index(['date','symbol'])

In [151]:
df_all_test

open        high        low  \
date                      symbol                                         
2019-07-20 00:00:00+00:00 BTC-PERP   10564.2500  11108.7500  10385.250   
                          BNB-PERP      30.7225     31.5825     29.135   
                          ETH-PERP     225.9300    235.5200    221.630   
2019-07-21 00:00:00+00:00 BTC-PERP   10765.7500  10834.0000  10322.000   
                          BNB-PERP      31.1425     31.5525     29.400   
...                                         ...         ...        ...   
2022-04-06 00:00:00+00:00 BNB-PERP     444.5575    444.6300    423.220   
                          LUNA-PERP    116.1980    118.3230    108.581   
                          SOL-PERP     126.8650    127.4450    116.525   
                          BTC-PERP   45504.0000  45515.0000  43775.000   
                          AVAX-PERP     91.0320     91.0640     83.137   

                                          close       pct        EMA_10  \
date                      symbol                                          
2019-07-20 00:00:00+00:00 BTC-PERP   10765.7500  0.000000           NaN   
                          BNB-PERP      31.1725  0.000000           NaN   
                          ETH-PERP     228.7600  0.000000           NaN   
2019-07-21 00:00:00+00:00 BTC-PERP   10588.7500 -0.016578           NaN   
                          BNB-PERP      30.4275 -0.024189           NaN   
...                                         ...       ...           ...   
2022-04-06 00:00:00+00:00 BNB-PERP     427.5475 -0.039014    435.399480   
                          LUNA-PERP    109.4960 -0.059408    108.672262   
                          SOL-PERP     117.8700 -0.073541    122.136461   
                          BTC-PERP   44302.0000 -0.026770  45510.877500   
                          AVAX-PERP     85.6250 -0.061234     91.939407   

                                           EMA_20        EMA_60     rsi_14  \
date                      symbol                                             
2019-07-20 00:00:00+00:00 BTC-PERP            NaN           NaN        NaN   
                          BNB-PERP            NaN           NaN        NaN   
                          ETH-PERP            NaN           NaN        NaN   
2019-07-21 00:00:00+00:00 BTC-PERP            NaN           NaN        NaN   
                          BNB-PERP            NaN           NaN        NaN   
...                                           ...           ...        ...   
2022-04-06 00:00:00+00:00 BNB-PERP     424.791643    415.267124  53.530126   
                          LUNA-PERP    102.673290     87.788183  59.856261   
                          SOL-PERP     113.654216    107.787650  56.578188   
                          BTC-PERP   44629.204817  43022.638361  51.117104   
                          AVAX-PERP     89.410023     84.822951  47.091455   

                                        roc_14     roc_30  low_of_52week  \
date                      symbol                                           
2019-07-20 00:00:00+00:00 BTC-PERP         NaN        NaN            NaN   
                          BNB-PERP         NaN        NaN            NaN   
                          ETH-PERP         NaN        NaN            NaN   
2019-07-21 00:00:00+00:00 BTC-PERP         NaN        NaN            NaN   
                          BNB-PERP         NaN        NaN            NaN   
...                                        ...        ...            ...   
2022-04-06 00:00:00+00:00 BNB-PERP    4.667915  12.462928       317.0950   
                          LUNA-PERP  15.380400  40.839925        10.7400   
                          SOL-PERP   24.214242  44.377756        31.2525   
                          BTC-PERP    3.248811  16.608760     35072.0000   
                          AVAX-PERP  -0.434889  19.471187        12.4500   

                                     high_of_52week  
date                      symbol              

In [150]:
df_all_test.loc[(df_all_test.index.get_level_values('symbol') == 'BTC-PERP') & (df_all_test.index.get_level_values('date') >= '2022-04-06')]

,,open,high,low,close,pct,EMA_10,EMA_20,EMA_60,rsi_14,roc_14,roc_30,low_of_52week,high_of_52week
date,symbol,,,,,,,,,,,,,
2022-04-06 00:00:00+00:00,BTC-PERP,45504.0,45515.0,43775.0,44302.0,-0.02677,45510.8775,44629.204817,43022.638361,51.117104,3.248811,16.60876,35072.0,67657.0


In [149]:
df_all_test.index.get_level_values('date')

DatetimeIndex(['2019-07-20 00:00:00+00:00', '2019-07-20 00:00:00+00:00',
               '2019-07-20 00:00:00+00:00', '2019-07-21 00:00:00+00:00',
               '2019-07-21 00:00:00+00:00', '2019-07-21 00:00:00+00:00',
               '2019-07-22 00:00:00+00:00', '2019-07-22 00:00:00+00:00',
               '2019-07-22 00:00:00+00:00', '2019-07-23 00:00:00+00:00',
               ...
               '2022-04-05 00:00:00+00:00', '2022-04-05 00:00:00+00:00',
               '2022-04-05 00:00:00+00:00', '2022-04-05 00:00:00+00:00',
               '2022-04-06 00:00:00+00:00', '2022-04-06 00:00:00+00:00',
               '2022-04-06 00:00:00+00:00', '2022-04-06 00:00:00+00:00',
               '2022-04-06 00:00:00+00:00', '2022-04-06 00:00:00+00:00'],
              dtype='datetime64[ns, UTC]', name='date', length=4570, freq=None)

### Format


In [162]:
df_all_test2=  df_all_test.unstack()

In [177]:
df_all_test2['close'].iloc[-3:]

symbol,AVAX-PERP,BNB-PERP,BTC-PERP,ETH-PERP,LUNA-PERP,SOL-PERP
date,,,,,,
2022-04-04 00:00:00+00:00,96.179,447.5275,46613.0,3521.7,116.478,132.150
2022-04-05 00:00:00+00:00,91.032,444.5575,45504.0,3406.5,116.198,126.865
2022-04-06 00:00:00+00:00,85.625,427.5475,44302.0,3266.2,109.496,117.870


In [170]:
df_all_test2['rsi_14'].iloc[[-1]]

symbol,AVAX-PERP,BNB-PERP,BTC-PERP,ETH-PERP,LUNA-PERP,SOL-PERP
date,,,,,,
2022-04-06 00:00:00+00:00,47.091455,53.530126,51.117104,55.265293,59.856261,56.578188



# df_all , df_all_test ,df_all_test2 ควรใช้ DataFrame แบบไหนมากกว่ากันครับ


# Test ZONE

In [65]:

df_all = pd.DataFrame()
df1 = get_data(symbol_lists[0]).drop(columns='volume')
df1['symbol'] = symbol_lists[0]
df1.ta.ema(200,append=True)
df1['ADX_20'] =df1.ta.adx(20)['ADX_20']
df1.ta.rsi(20,append=True)
df1.ta.roc(60,append=True)

df2 = get_data(symbol_lists[1]).drop(columns='volume')
df2['symbol'] = symbol_lists[1]
df2.ta.ema(200,append=True)
df2['ADX_20'] =df2.ta.adx(20)['ADX_20']
df2.ta.rsi(20,append=True)
df2.ta.roc(60,append=True)

df_all =df_all.append([df1,df2])
df_all = df_all.reset_index().set_index('symbol').sort_values('date')

In [126]:
df_all

,date,open,high,low,close,pct,EMA_10,EMA_20,EMA_60,rsi_14,roc_14,roc_30,low_of_52week,high_of_52week
symbol,,,,,,,,,,,,,,
BTC-PERP,2019-07-20 00:00:00+00:00,10564.250,11108.750,10385.250,10765.750,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BTC-PERP,2019-07-21 00:00:00+00:00,10765.750,10834.000,10322.000,10588.750,-0.016578,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BTC-PERP,2019-07-22 00:00:00+00:00,10588.750,10688.250,10059.000,10321.500,-0.025563,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BTC-PERP,2019-07-23 00:00:00+00:00,10321.500,10323.250,9776.500,9841.750,-0.047596,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BTC-PERP,2019-07-24 00:00:00+00:00,9835.250,9915.750,9517.750,9770.000,-0.007317,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AVAX-PERP,2022-04-02 00:00:00+00:00,96.861,103.714,95.720,96.258,-0.006245,92.286288,88.039142,83.720490,64.708570,7.430804,22.687298,11.255,134.978
AVAX-PERP,2022-04-03 00:00:00+00:00,96.258,100.180,94.399,98.081,0.018762,93.339872,88.995510,84.191326,66.504106,15.575693,29.457651,11.465,134.978
AVAX-PERP,2022-04-04 00:00:00+00:00,98.081,98.131,90.709,96.179,-0.019583,93.856077,89.679652,84.584365,62.907948,11.540335,25.486333,11.936,134.978
